### Create a kernel and add a chat completion model to it 

In [1]:
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = Kernel()

# Auto-loads defaults from .env file, alternatively you can set endpoint, deployment_name and api_key directly
chat_completion = AzureChatCompletion()

kernel.add_service(chat_completion)

In [9]:
!pip install psycopg2

  Using cached psycopg2-2.9.10-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
Using cached psycopg2-2.9.10-cp311-cp311-win_amd64.whl (1.2 MB)


### Provide the kernel with skills to use. These can be defined as "functions" and "plugins"

##### Define function via a prompt (semantic function)

In [3]:
prompt_template = "{{$input}}\n\ntranslate to {{$target_language}} and change the tone to {{$target_tone}}:"

rewrite_expert = kernel.add_function(
    prompt=prompt_template,
    function_name="toner",
    plugin_name="Toner"
)

# Use the function
sample_text = """
Semantic Kernel is a lightweight, open-source development kit that lets 
you easily build AI agents and integrate the latest AI models into your codebase
"""

summary = await kernel.invoke(rewrite_expert, input=sample_text, target_language = "english", target_tone="sarcastic")
print(summary)

Sure! Here’s your sentence translated to English with a sarcastic tone:

Oh, fantastic—Semantic Kernel is just that “lightweight” open-source toolkit that *supposedly* makes it super easy to build AI agents and jam the latest AI models right into your precious codebase.


##### Toy example: Define function via code 

In [4]:
# importing all we need for this session
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from typing import TypedDict, Annotated, List, Optional
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.function_choice_behavior import (
    FunctionChoiceBehavior,
)
from semantic_kernel.contents.chat_history import ChatHistory

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)



In [5]:
class LightModel(TypedDict):
    id: int
    name: str
    is_on: bool | None
    brightness: int | None
    hex: str | None


class LightsPlugin:
    def __init__(self, lights: list[LightModel]):
        self.lights = lights

    @kernel_function
    async def get_lights(self) -> List[LightModel]:
        """Gets a list of lights and their current state."""
        return self.lights

    @kernel_function
    async def get_state(
        self, id: Annotated[int, "The ID of the light"]
    ) -> Optional[LightModel]:
        """Gets the state of a particular light."""
        for light in self.lights:
            if light["id"] == id:
                return light
        return None

    @kernel_function
    async def change_state(
        self, id: Annotated[int, "The ID of the light"], new_state: LightModel
    ) -> Optional[LightModel]:
        """Changes the state of the light."""
        for light in self.lights:
            if light["id"] == id:
                light["is_on"] = new_state.get("is_on", light["is_on"])
                light["brightness"] = new_state.get("brightness", light["brightness"])
                light["hex"] = new_state.get("hex", light["hex"])
                return light
        return None

In [6]:
# Create dependencies for the plugin
# Example with toy data in memory
lights = [
    {"id": 1, "name": "Table Lamp", "is_on": False, "brightness": 100, "hex": "FF0000"},
    {"id": 2, "name": "Porch light", "is_on": False, "brightness": 50, "hex": "00FF00"},
    {"id": 3, "name": "Chandelier", "is_on": True, "brightness": 75, "hex": "0000FF"},
]

plugin = LightsPlugin(lights=lights)

kernel.add_plugin(
    plugin=plugin,
    plugin_name="Lights",
)


KernelPlugin(name='Lights', description=None, functions={'change_state': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='change_state', plugin_name='Lights', description='Changes the state of the light.', parameters=[KernelParameterMetadata(name='id', description='The ID of the light', default_value=None, type_='int', is_required=True, type_object=<class 'int'>, schema_data={'type': 'integer', 'description': 'The ID of the light'}, include_in_function_choices=True), KernelParameterMetadata(name='new_state', description=None, default_value=None, type_='LightModel', is_required=True, type_object=<class '__main__.LightModel'>, schema_data={'type': 'object', 'properties': {'id': {'type': 'integer'}, 'name': {'type': 'string'}, 'is_on': {'type': ['boolean', 'null']}, 'brightness': {'type': ['integer', 'null']}, 'hex': {'type': ['string', 'null']}}, 'required': ['id', 'name']}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=True, return_parameter=KernelPa

In [7]:
# Enable planning
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Create a history of the conversation
history = ChatHistory()
user_message = "Please turn on the lamp"
history.add_user_message(user_message)

while(True):
    # Get the response from the AI
    
    result = await chat_completion.get_chat_message_content(
        chat_history=history,
        settings=execution_settings,
        kernel=kernel,
    )

    # Print the results
    print("Assistant > " + str(result))
    if(user_message.lower() == "exit"):
        print("Exiting the chat. Goodbye!")
        break
    # Add the message from the agent to the chat history
    history.add_message(result)

    # Get user input for the next message
    user_message = input("Type another request or say 'exit' to end the chat > ")
    history.add_user_message(user_message)
    if( user_message.lower() == "exit"):
        #autiomatic function calling
        history.add_user_message("Please turn off all the lamps and give me final state of all the lights")




Assistant > The lamp (Table Lamp) has been turned on. If you need any adjustments to the brightness or color, just let me know!
Assistant > All lamps have been turned off. Here is the final state of all the lights:

1. Table Lamp: Off, Brightness 100, Color FF0000
2. Porch Light: Off, Brightness 50, Color 00FF00
3. Chandelier: Off, Brightness 75, Color 0000FF

Let me know if you need any other changes!
Exiting the chat. Goodbye!


### A real world example: ABC Electronics Ltd.
ABC Electronics Ltd. is an electronic devices distribution company that sells a large variety of devices (ex. headphones, laptops, TVs, etc.) new and refurbished.

##### First, let's set up the database for company ABC (one time)
- You need an Azure postgres db (database name is abc)
- Add credentials to .env file
- Uncomment and run below python script


In [10]:
# Connect to database via Microsoft Entra authentication
!python db_init.py


Connection uri was rertieved successfully.
Successfully Connected to - ('PostgreSQL 16.8 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 11.2.0, 64-bit',)
Executing SQL file: ABC_co.sql
-- Drop Tables if they exist
DROP TABLE IF EXISTS reviews CASCADE;
DROP TABLE IF EXISTS shipments CASCADE;
DROP TABLE IF EXISTS return_items CASCADE;
DROP TABLE IF EXISTS sales CASCADE;
DROP TABLE IF EXISTS customers CASCADE;
DROP TABLE IF EXISTS products CASCADE;

-- Create Tables
CREATE TABLE products (
    product_id SERIAL PRIMARY KEY,
    name VARCHAR(150) NOT NULL,
    description TEXT,
    price NUMERIC(10, 2),
    inventory NUMERIC(10, 2),
    refurbished BOOLEAN DEFAULT FALSE,
    category VARCHAR(50) DEFAULT 'Eelectronics'
);
CREATE TABLE customers(
    customer_id SERIAL PRIMARY KEY,
    city VARCHAR(100),
    state VARCHAR(100),
    country VARCHAR(100),
    sentiment_score NUMERIC(5, 2) DEFAULT 0,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL
);
CREATE TABLE sales (
 

#### **Example 1**: an inventory analyst working at the sales departments, wants to get below insights based on current inventory and sales data of the company:
- Most updated inventory for a given product
- A list of most sold product names for a given categroy
- Adding a new product to the database

This data is stored in a postgreSQL database.
In this example, we take a simple approach as follows:
1- Connect to the database
2- Load data from required tables from the database
3- Provide all necessary functions and plugins
4- Develop a code using semantic kernel to get the answers

In [12]:
import psycopg2
from get_conn import get_connection_uri
from pandas import DataFrame
# from typing import List, Tuple


conn_uri = get_connection_uri()

class ABC_DataPlugin:
    def __init__(self, db_uri: str):
        self.conn = psycopg2.connect(db_uri)
        self.cursor = self.conn.cursor()
        print("Connected to company's database successfully.")

    @kernel_function
    async def get_product_info(self, product_name: Optional[str] = None, product_id: Optional[int] = None) -> list[dict]:
        """Gets all product information from the database."""
        query = """SELECT 
                    product_id,                   
                    name,
                    inventory,
                    price,
                    refurbished,
                    category
                FROM products
                WHERE (LOWER(name) = LOWER(%(product_name)s) AND %(product_name)s IS NOT NULL)
                   OR (product_id = %(product_id)s AND %(product_id)s IS NOT NULL)
                   """
        if not product_name and not product_id:
            print("No valid product name or ID provided.")
            return None
        elif product_id:
            self.cursor.execute(query, {"product_name": None, "product_id": product_id})
        else:
            self.cursor.execute(query, {"product_name": product_name, "product_id": None})

            
        rows = self.cursor.fetchall()
        columns = [desc[0] for desc in self.cursor.description]
        try:
            products= DataFrame(rows, columns=columns)
            products.to_dict(orient="records")  # <-- JSON serializabl
            
            return products.to_dict(orient="records")  # <-- JSON serializabl
        except Exception as e:
            print(f"Error fetching product information: {e}")
            return None
    @kernel_function
    def most_sold_product(self, category: str) -> Optional[dict]:
        """Returns the most sold product in a given category."""
        query = """
            SELECT products.product_id, products.name, SUM(sales.quantity) AS total_sold
            FROM sales
            JOIN products ON sales.product_id = products.product_id
            WHERE LOWER(products.category) = LOWER(%(category)s) 
            GROUP BY products.product_id, products.name
            ORDER BY total_sold DESC
            LIMIT 1;
        """
        self.cursor.execute(query, {"category": category})
        row = self.cursor.fetchone()
        if row:
            columns = [desc[0] for desc in self.cursor.description]
            return dict(zip(columns, row))
        else:
            print(f"No sales data found for category: {category}")
            return None
    @kernel_function
    async def add_product(self, name: str, description: str, price: float,
                           inventory: int, refurbished: bool, category: str ) -> bool:
        """Adds a new product to the database."""
        query = """INSERT INTO products (name, description, price, inventory, refurbished, category)
                   VALUES (%(name)s, %(description)s, %(price)s, %(inventory)s, %(refurbished)s, %(category)s);"""  
        try:
            self.cursor.execute(
                query,
                {
                    "name": name,
                    "description": description,
                    "price": price,
                    "inventory": inventory,
                    "refurbished": refurbished,
                    "category": category
                }
            )
            self.conn.commit()
            print(f"Product '{name}' added successfully.")
            return True
        except Exception as e:
            print(f"Error adding product: {e}")
            self.conn.rollback()
            return False

    def close_connection(self):
        """Closes the database connection."""
        self.cursor.close()
        self.conn.close()
        print("Database connection closed.")
    



Connection uri was rertieved successfully.


In [13]:
abc_kernel = Kernel()

# Auto-loads defaults from .env file, alternatively you can set endpoint, deployment_name and api_key directly
chat_completion = AzureChatCompletion()
abc_kernel.add_service(chat_completion)

abc_plugin = ABC_DataPlugin(db_uri=conn_uri)

abc_kernel.add_plugin(
    plugin=abc_plugin,
    plugin_name="ABC_Plugin"
) 

# Enable planning
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Create a history of the conversation
history = ChatHistory()


Connected to company's database successfully.


In [14]:
user_message = "Please give me information about the product with id 1"
history.add_user_message(user_message)

# Get the response from the AI
    
result = await chat_completion.get_chat_message_content(
    chat_history=history,
    settings=execution_settings,
    kernel=abc_kernel,
)
print("Assistant > " + str(result))

Assistant > The product with ID 1 is the Macbook Pro M4. Here are its details:

- Category: Laptops
- Price: $2085.00
- Inventory: 50 units available
- Refurbished: No (This is a new product)

Let me know if you need more information!
